In [9]:
!pip install -r requirements.txt

  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 22.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 22.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.5/767.5 kB 22.0 MB/s eta 0:00:00
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 25.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72/72 [jupyter]0/72 [notebook]b]server]minals]


In [12]:
path_in =r"/home/tm/Рабочий стол/ projects/etl_project/data/in.xlsx"

In [13]:
import pandas as pd
import re
from openpyxl import load_workbook
from openpyxl.utils import column_index_from_string

# Ваш существующий INDICATOR_MAPPING остается без изменений
INDICATOR_MAPPING = {
    # Добыча УВ сырья
    "1.1.": "ДОБЫЧА УВ СЫРЬЯ. Нефть, млн. т",
    "1.2.": "ДОБЫЧА УВ СЫРЬЯ. Конденсат, млн. т",
    "1.3.": "ДОБЫЧА УВ СЫРЬЯ. Газ (свободный + газовая шапка), млрд. м3",
    "1.4.": "ДОБЫЧА УВ СЫРЬЯ. Попутный газ, млрд. м3",
    
    # Затраты на ГРР
    "2.": "ЗАТРАТЫ НА ГРР. ВСЕГО*, млн. р.",
    "2.1.": "Глубокое бурение. всего, млн. р.",
    "2.1.1.": "Параметрическое, млн. р.",
    "2.1.2.": "Поисковое, млн. р.",
    "2.1.3.": "Разведочное, млн. р.",
    "2.2.": "Сейсморазведка. всего, млн. р.",
    "2.2.1.": "Сейсморазведка 2Д, млн. р.",
    "2.2.2.": "Сейсморазведка 3Д, млн. р.",
    "2.3.": "НИОКР, млн. р.",
    "2.4.": "Прочие работы, млн. р.",
    
    # Результаты ГРР
    "3.1.": "Глубокое бурение. всего, м",
    "3.1.1.": "Параметрическое, м",
    "3.1.2.": "Поисковое, м",
    "3.1.3.": "Разведочное, м",
    "3.1.4.": "Закончено строительством поисковых скважин, шт.",
    "3.1.4.1.": "Из них продуктивных, шт.",
    "3.1.5.": "Закончено строительством разведочных скважин, шт.",
    "3.1.5.1.": "Из них продуктивных, шт.",
    
    # Прирост запасов АВ1C1
    "3.2.1.": "Прирост запасов УВ категории АВ1C1. Нефть, млн. т",
    "3.2.1.1.": "Прирост запасов УВ категории АВ1C1. Нефть за счет ГРР, млн. т",
    "3.2.1.2.": "Прирост запасов УВ категории АВ1C1. Нефть за счет переоценки, млн. т",
    "3.2.2.": "Прирост запасов УВ категории АВ1C1. Конденсат, млн. т",
    "3.2.2.1.": "Прирост запасов УВ категории АВ1C1. Конденсат за счет ГРР, млн. т",
    "3.2.2.2.": "Прирост запасов УВ категории АВ1C1. Конденсат за счет переоценки, млн. т",
    "3.2.3.": "Прирост запасов УВ категории АВ1C1. Газ (свободный + газовая шапка), млрд. м3",
    "3.2.3.1.": "Прирост запасов УВ категории АВ1C1. Газ за счет ГРР, млрд. м3",
    "3.2.3.2.": "Прирост запасов УВ категории АВ1C1. Газ за счет переоценки, млрд. м3",
    "3.2.4.1.": "Прирост запасов УВ категории АВ1C1. Нефть на вновь открытых месторождениях, млн. т",
    "3.2.4.2.": "Прирост запасов УВ категории АВ1C1. Конденсат на вновь открытых месторождениях, млн. т",
    "3.2.4.3.": "Прирост запасов УВ категории АВ1C1. Газ на вновь открытых месторождениях, млрд. м3",
    
    # Прирост запасов В2С2com
    "3.3.1.": "Прирост запасов УВ категории В2С2. Нефть, млн. т",
    "3.3.1.1.": "Прирост запасов УВ категории В2С2. Нефть за счет ГРР, млн. т",
    "3.3.1.2.": "Прирост запасов УВ категории В2С2. Нефть за счет переоценки, млн. т",
    "3.3.2.": "Прирост запасов УВ категории В2С2. Конденсат, млн. т",
    "3.3.2.1.": "Прирост запасов УВ категории В2С2. Конденсат за счет ГРР, млн. т",
    "3.3.2.2.": "Прирост запасов УВ категории В2С2. Конденсат за счет переоценки, млн. т",
    "3.3.3.": "Прирост запасов УВ категории В2С2. Газ (свободный + газовая шапка), млрд. м3",
    "3.3.3.1.": "Прирост запасов УВ категории В2С2. Газ за счет ГРР, млрд. м3",
    "3.3.3.2.": "Прирост запасов УВ категории В2С2. Газ за счет переоценки, млрд. м3",
    "3.3.4.1.": "Прирост запасов УВ категории В2С2. Нефть на вновь открытых месторождениях, млн. т",
    "3.3.4.2.": "Прирост запасов УВ катеcomгории В2С2. Конденсат на вновь открытых месторождениях, млн. т",
    "3.3.4.3.": "Прирост запасов УВ категории В2С2. Газ на вновь открытых месторождениях, млрд. м3",
    
    # Нетрадиционные источники УВС
    "3.4.1.": "Нетрадиционные источники УВС. Прирост категории АВ1С1. Сланцевый газ, млрд. м3",
    "3.4.2.": "Нетрадиционные источники УВС. Прирост категории АВ1С1. Сланцевая нефть, млн. т",
    "3.5.1.": "Нетрадиционные источники УВС. Прирост категории С2 (В2). Сланцевый газ, млрд. м3",
    "3.5.2.": "Нетрадиционные источники УВС. Прирост категории С2 (В2). Сланцевая нефть, млн. т",
    
    # Сейсморазведка
    "3.6.1.": "Сейсморазведка 2Д, пог. км",
    "3.6.2.": "Сейсморазведка 3Д, км2",
    
    # Подготовлено сейсморазведкой перспективных объектов
    "3.7.1.": "Подготовлено сейсморазведкой перспективных объектов с ресурсами. Нефть, шт.",
    "3.7.1.1": "Подготовлено сейсморазведкой перспективных объектов с ресурсами. Нефть, млн. т",
    "3.7.2.": "Подготовлено сейсморазведкой перспективных объектов с ресурсами. Газ (конденсат), шт.",
    "3.7.2.1": "Подготовлено сейсморазведкой перспективных объектов с ресурсами. Газ, млрд. м3",
    "3.7.2.2": "Подготовлено сейсморазведкой перспективных объектов с ресурсами. Конденсат, млн. т",
    
    # Выведено (списано) по результатам бурения
    "3.8.1.": "Выведено (списано) по результатам бурения поисковых объектов. Нефть, шт.",
    "3.8.1.1": "Выведено (списано) по результатам бурения поисковых объектов. Нефть, млн. т",
    "3.8.2.": "Выведено (списано) по результатам бурения поисковых объектов. Газ (конденсат), шт.",
    "3.8.2.1": "Выведено (списано) по результатам бурения поисковых объектов. Газ, млрд. м3",
    "3.8.2.2": "Выведено (списано) по результатам бурения поисковых объектов. Конденсат, млн. т",
    
    # Состояние ресурсов
    "3.9.1.": "Состояние ресурсов кат. Д0 на 01.01.2023 г. Нефть, млн. т",
    "3.9.2.": "Состояние ресурсов кат. Д0 на 01.01.2023 г. Газ, млрд. м3",
    "3.9.3.": "Состояние ресурсов кат. Д0 на 01.01.2023 г. Конденсат, млн. т",
    
    # Открыто месторождений
    "3.10.": "Открыто месторождений, всего, шт.",
    "3.10.1.": "Открыто месторождений, нефтяных, шт.",
    "3.10.2.": "Открыто месторождений, газовых, шт.",
    "3.10.3.": "Открыто месторождений, нефтегазовых и газонефтяных, шт.",
    "3.10.4.": "Открыто месторождений, нефтегазоконденсатных и газоконденсатных, шт.",
    
    # Оценка выполнения лицензионных условий
    "4.": "Оценка выполнения лицензионных условий, балл"
}
REGION_NORMALIZATION = {
    r'хмао[\s\-]?югр[аы]': 'ХМАО — Югра',
    r'ханты[\s\-]мансийск[\w\s\-]*югр[аы]': 'ХМАО — Югра',
    r'ямал[\w\s\-]*округ': 'Ямало-Ненецкий АО',
    r'ямальск[\w\s\-]*район': 'Ямало-Ненецкий АО',
    r'янао': 'Ямало-Ненецкий АО'
}
def normalize_region(region_name):
    """Приводит название региона к стандартному виду"""
    if not region_name or not isinstance(region_name, str):
        return None
    
    # Приводим к нижнему регистру и удаляем лишние символы
    cleaned = re.sub(r'[^\w\s\-]', '', region_name.lower().strip())
    
    # Ищем совпадения с нашим словарем
    for pattern, standard in REGION_NORMALIZATION.items():
        if re.search(pattern, cleaned):
            return standard
    
    # Если не нашли совпадение, возвращаем оригинал с базовой очисткой
    return region_name.strip()
def extract_company(text):
    # Нормализация текста
    text = re.sub(r'_{2,}', ' ', text)  # Заменяем множественные подчеркивания
    text = re.sub(r'\s+', ' ', text)  # Заменяем множественные пробелы
    text = text.replace('«', '"').replace('»', '"')  # Стандартизируем кавычки
    
    # Улучшенный паттерн поиска
    pattern = r'''
        (ПАО|АО|ООО|ЗАО|НКО)  # Тип организации
        \s*                    # Возможные пробелы
        "                      # Открывающая кавычка
        (                      # Начинаем захват названия
          (?:                  # Группа без захвата
            [^"\n]+            # Любые символы кроме кавычки и переноса строки
            (?:"[^"\n]+)*      # Вложенные кавычки с текстом
          )                    #
        )                      # Конец названия
        "                      # Закрывающая кавычка
        (?!\S)                 # Не должно быть букв/цифр после
    '''
    
    match = re.search(pattern, text, re.VERBOSE | re.IGNORECASE)
    if match:
        company_type = match.group(1).upper()
        company_name = match.group(2)
        
        # Очистка названия (без обрезания существенной части)
        clean_name = company_name.strip(' _-')
        return f'{company_type} "{clean_name}"'
    
    # Дополнительный поиск для случаев с большим количеством пробелов
    alt_pattern = r'(ПАО|АО|ООО|ЗАО|НКО)\s+"([^"]+)"'
    alt_match = re.search(alt_pattern, text, re.IGNORECASE)
    if alt_match:
        return f'{alt_match.group(1).upper()} "{alt_match.group(2).strip()}"'
    
    return None
def cut_company_name(company):
    company = company.split()
    if len(company)>=3:
        company=company[:3]
    return " ".join(company)

def extract_metadata(sheet):
    """Извлекает метаданные из листа"""
    metadata = {
        'subject':None,
        'company': None,
        'license': None,
        'area': None,
        'vink': None
    }
    
    # Поиск названия компании
    for row in sheet.iter_rows(values_only=True):
        if not any(row):
            continue
        
        row_text = ' '.join(str(cell) for cell in row if cell).replace('\n', ' ')
        
        if "(наименование компании)" in row_text or "выполненных в" in row_text :
            match = extract_company(row_text)
            if match:
                metadata['company'] = cut_company_name(match)
            break
    for row in sheet.iter_rows(max_row=15, values_only=True):
        if not any(row):
            continue
            
        row_text = ' '.join(str(cell) for cell in row if cell)
        if not row_text:
            continue
            
        # Проверяем типичные упоминания региона
        if any(word in row_text.lower() for word in ['хмао', 'ханты', 'югра', 'ямал', 'янао']):
            metadata['subject'] = normalize_region(row_text)
            break
    # Поиск лицензии и ВИНК
    license_pattern = re.compile(r'Лицензия\s*([А-Я]{2,}\s*\d+\s*[А-Я]{2,})[\s,]*([А-Яа-я\-\s]+)')
    vink_pattern = re.compile(
    r'ВИНК\*\s*((?:ПАО|АО|ООО|ЗАО|НКО)\s*)?'  # Орг. форма (опционально, сохраняем)
    r'("[^"]*(?:"[^"]*)*"|[^"\s]+)'  # Название (с кавычками или без)
    r'(?=\s*(?:доля|$|\n))',  # Стоп-символы
    flags=re.IGNORECASE
)
      
    for row in sheet.iter_rows(values_only=True):
        if not any(row):
            continue
        row_text = ' '.join(str(cell) for cell in row if cell)
        
        # Поиск лицензии
        license_match = license_pattern.search(row_text)
        if license_match and not metadata['license']:
            metadata['license'] = license_match.group(1).strip()
            metadata['area'] = license_match.group(2).strip().replace(" Шельфовое продолжение",'')
        
        # Поиск ВИНК
        vink_match = vink_pattern.search(row_text)
        if vink_match:
            # Объединяем все непустые группы через пробел
            combined = ' '.join(group for group in vink_match.groups() if group)
            metadata['vink'] = combined
    
    return metadata

def process_workbook(file_path):
    wb = load_workbook(file_path,data_only=True)
    all_data = []
    i=1
    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]
        
        # Извлекаем метаданные один раз для листа
        metadata = extract_metadata(sheet)
        
        # Находим колонки с данными
        fact_col = plan_col = None
        for row in sheet.iter_rows(max_row=10):
            for cell in row:
                if cell.value and isinstance(cell.value, str):
                    text = str(cell.value).lower()
                    if "2022" in text and "факт" in text:
                        fact_col = cell.column_letter
                    elif "2023" in text and "план" in text:
                        plan_col = cell.column_letter
        
        if not fact_col or not plan_col:
            print(f"Не найдены колонки с данными в листе: {sheet_name}")
            continue
        
        # Сначала собираем ВСЕ показатели за 2022 год
        records_2022 = {
            'ID':i,
            'Субъект РФ': metadata.get("subject"),
            'Период (год)': 2022,
            'план/факт':'факт',
            'Главная компания': metadata.get('company'),
            'Недропользователь': metadata.get('vink'),
            '№ лицензии': metadata.get('license'),
            'участок': metadata.get('area'),
            'Оценка выполнения лицензионных условий': None
        }
        i+=1
        
        # Затем ВСЕ показатели за 2023 год
        records_2023 = {
            'ID':i,
            'Субъект РФ': metadata.get("subject"),
            'Период (год)': 2023,
            'план/факт': 'план',
            'Главная компания': metadata.get('company'),
            'Недропользователь': metadata.get('vink'),
            '№ лицензии': metadata.get('license'),
            'участок': metadata.get('area'),
            'Оценка выполнения лицензионных условий': None
        }
        i+=1
        # Заполняем показатели
        for row in sheet.iter_rows(min_row=2, values_only=True):
            if not row or not isinstance(row[0], str):
                continue
                
            p_p = row[0].strip()
            if p_p in INDICATOR_MAPPING:
                col_name = INDICATOR_MAPPING[p_p]
                records_2022[col_name] = row[column_index_from_string(fact_col)-1]
                records_2023[col_name] = row[column_index_from_string(plan_col)-1]
        
        # Добавляем в общий список
        all_data.append(records_2022)
        all_data.append(records_2023)
    
    # Создаем DataFrame
    df = pd.DataFrame(all_data)
    
    # Заполняем None для отсутствующих показателей
    for col in INDICATOR_MAPPING.values():
        if col not in df.columns:
            df[col] = None
    df.iloc[:, 8] = df.iloc[:, 72]

# Удаление колонки 'BU'
    df.drop(df.columns[72], axis=1, inplace=True)
    return df


# Сохранение в Excel и CSV
def save_results(df, base_path):
    # Сохранение в Excel
    excel_path = f"{base_path}/результат.xlsx"
    df.to_excel(excel_path, index=False)
    
    # Сохранение в CSV
    csv_path = f"{base_path}/результат.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')


In [14]:
#запуск ф-ий
out_path =r'/home/tm/Рабочий стол/ projects/etl_project/results' 
result_df = process_workbook(path_in)
save_results(result_df,out_path)    


In [ ]:
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Параметры подключения к PostgreSQL (админские права)
admin_db_config = {
    'host': '127.0.0.1',
    'port': '5432',
    'user': 'postgres', 
    'password': '123',  # Администратор БД
    'database': 'postgres'  # Подключаемся к системной БД
}

# Параметры для новой БД
new_db_name = "oil_gas_db"
new_user = "oil_analyst"
new_password = "pass"

try:
    # Подключаемся к PostgreSQL
    conn = psycopg2.connect(**admin_db_config)
    
    # Устанавливаем режим autocommit на уровне соединения
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    
    with conn.cursor() as cur:
        # 1. Создаем базу данных
        cur.execute(sql.SQL("CREATE DATABASE {}").format(
            sql.Identifier(new_db_name)))
        print(f"База данных {new_db_name} создана")
        
        # 2. Создаем пользователя
        cur.execute(sql.SQL("CREATE USER {} WITH PASSWORD %s").format(
            sql.Identifier(new_user)), [new_password])
        print(f"Пользователь {new_user} создан")
        
        # 3. Даем права на базу данных
        cur.execute(sql.SQL("GRANT ALL PRIVILEGES ON DATABASE {} TO {}").format(
            sql.Identifier(new_db_name),
            sql.Identifier(new_user)))
        print("Базовые права на БД назначены")
        
        # 4. Даем права на схему public
        cur.execute(sql.SQL("GRANT ALL PRIVILEGES ON SCHEMA public TO {}").format(
            sql.Identifier(new_user)))
        print("Права на схему public выданы")
        
        # 5. Разрешаем создание временных таблиц
        cur.execute(sql.SQL("ALTER USER {} CREATEDB").format(
            sql.Identifier(new_user)))
        print("Право CREATEDB выдано")
        
        # 6. Разрешаем создание объектов в БД
        cur.execute(sql.SQL("GRANT CREATE ON DATABASE {} TO {}").format(
            sql.Identifier(new_db_name),
            sql.Identifier(new_user)))
        print("Право CREATE выдано")

except psycopg2.Error as e:
    print(f"Ошибка PostgreSQL: {e}")
finally:
    if 'conn' in locals():
        conn.close()

Базовые права на БД назначены
Права на схему public выданы
Право CREATEDB выдано
Право CREATE выдано


In [23]:
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Подключение как администратор
admin_conn = psycopg2.connect(
    host='localhost',
    port='5432',
    user='postgres',
    password='123',  # ваш пароль postgres
    database='postgres'
)
admin_conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

try:
    with admin_conn.cursor() as admin_cur:
        # Даем все права на базу данных
        admin_cur.execute(sql.SQL("""
            GRANT ALL PRIVILEGES ON DATABASE oil_gas_db TO oil_analyst;
            GRANT ALL PRIVILEGES ON SCHEMA public TO oil_analyst;
            GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA public TO oil_analyst;
            ALTER DEFAULT PRIVILEGES IN SCHEMA public 
            GRANT ALL PRIVILEGES ON TABLES TO oil_analyst;
        """))
        print("Все права успешно предоставлены")
finally:
    admin_conn.close()

Все права успешно предоставлены


In [24]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd


DB_CONFIG = {
    'host': 'localhost',
    'port': '5432',
    'dbname': 'oil_gas_db',
    'user': "oil_analyst",
    'password': "pass"
}

# Создаем подключение
engine = create_engine(f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}")
def generate_create_table_sql():
    """Генерирует SQL для создания таблицы с колонками для всех индикаторов"""
    columns = []
    
    # Базовые колонки (метаданные)
    base_columns = [
        'id SERIAL PRIMARY KEY',
        'subject VARCHAR(100)',
        'company VARCHAR(200)',
        'license_number VARCHAR(50)',
        'area VARCHAR(200)',
        'vink VARCHAR(200)',
        'year INTEGER',
        'plan_fact VARCHAR(10)',
        'license_evaluation FLOAT'
    ]
    
    # Колонки для индикаторов (тип FLOAT для числовых значений)
    indicator_columns = [f'"{indicator}" FLOAT' for indicator in INDICATOR_MAPPING.values()]
    
    return f"CREATE TABLE IF NOT EXISTS oil_gas_data ({', '.join(base_columns + indicator_columns)})"

def load_to_postgres(df):
    """Загружает данные в PostgreSQL с правильной структурой таблицы"""
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    # Создаем таблицу
    cur.execute(generate_create_table_sql())
    
    # Подготавливаем SQL для вставки
    columns = [
        'subject', 'company', 'license_number', 'area', 'vink', 
        'year', 'plan_fact', 'license_evaluation'
    ] + list(INDICATOR_MAPPING.values())
    
    placeholders = ', '.join(['%s'] * len(columns))
    sql = f"INSERT INTO oil_gas_data ({', '.join(columns)}) VALUES ({placeholders})"
    
    # Загружаем данные построчно
    for _, row in df.iterrows():
        values = []
        for col in columns:
            # Преобразуем None в NULL для PostgreSQL
            value = row[col] if pd.notna(row[col]) else None
            values.append(value)
        
        cur.execute(sql, values)
    
    conn.commit()
    print(f"Успешно загружено {len(df)} записей")
    cur.close()
    conn.close()



In [25]:
load_to_postgres(result_df)

InsufficientPrivilege: permission denied for schema public
LINE 1: CREATE TABLE IF NOT EXISTS oil_gas_data (id SERIAL PRIMARY K...
                                   ^


Алгоритм действий:<br>
установил необходимые библиотеки и импортировал их
указал полный путь к файлу из которого я буду получать данные 
все управление идет из основной функции process_workbook(file_path)
в которой вызываются остальные функции , которые помогают преобразовать данные к ним попорядку
так как в эксель файле есть формулы , я подключил книгу с атрибутом data_only=True, что позволяет в ячейках читать только значения 
for sheet_name in wb.sheetnames: - основной цикл в котором я прохожусь по всем листам эксель
metadata = extract_metadata(sheet) - первая надпись которая нас встречает она же первая функция в ней я получаю данные в словарь , это такие даные как :
            'Субъект РФ'   - в функции метадата пробегаю по листу и каждую строку в которой есть такие метки как  ['хмао', 'ханты', 'югра', 'ямал', 'янао'] вызываю функцию normalize_region в которой идет предобработка отчистка данных от личшних символов и идет сравнение с регуляркой которые перечислены в словаре REGION_NORMALIZATION и возвращается значение того ключа где регулярное выражение сработало, при желании как словарь , так и метки можно дополнять субъектами рф , список и словарь легко расширяется
            'Главная компания' - слов нет , насколько сложно вытаскивать компанию из заголовка, так же как и в субъекте вызывается функция extract_company где идет тройная предобработка даннх после я пытался разными способами с нейросетью вытянуть организацию, пока не подобрал слово к которому можно привязаться и в строке от которого скорее всего будет идти то самое наименование организации - это "(наименование компании)" , но оказалось все не так просто , на одном листе этого слова не было , пришлось цепляться еще к одной метке которая срабатывает в том случае условие выглядит так if "(наименование компании)" in row_text or "выполненных в" in row_text : 
            'Недропользователь' - так же вытягивал но уже боле простым способом из строки ВИНК*  - как я понял из логики таблицы
            '№ лицензии' - аналогичным способом 
            'участок' - идет рядом с лицензией , через запятую поэтому он тоже легко вытянулся
после заполренный словарь метадата я возвращаю в главную функцию 
дальше цикл я просто накожу имя колонки с показателям иза 2022 и за 2023 год , завожу переменную инициализирую единицей и после каждой строки увеличиваю значение - это для ID 
завожу два словаря records 2022 и  records 2023  в которые буду подставлять данные из словаря метадата и также некоторые поля таблицы мне надо чередовать в рамках каждой фирмы сначала 2022 потом 2023 поэтому два словаря 
в цикле имеется вот такая конструкция            
            ```p_p = row[0].strip() -получаю из пункта пп номер показателя которые у нас в первом столбце
            if p_p in INDICATOR_MAPPING: - выше вы видели словарь с перечислением всех показателей и тут мы просто если он есть 
                col_name = INDICATOR_MAPPING[p_p] - то присваем переменной имя колонки  и ниже заполняем два словаря по имени колонки  из   факта  и плана в соотвествующие словари
                records_2022[col_name] = row[column_index_from_string(fact_col)-1]
                records_2023[col_name] = row[column_index_from_string(plan_col)-1]```
после они построчно записываются в список all_data на основе которого потом строится датафрейм там я удалил лишний столбец который у меня повился , а еще я в метаданных когда формировал имя комании дописал функцию , чтобы лишние слова не записывались в имя организации , где строку сплитовал по пробелу и если больше трех желементов обрезал массив и возвращал потом заджоиную функцию.

тз довольно не простое в целом у меня не него ушло наверное 8 часов работы непрерывной, надеюсь я покрыл все требования к тз и правильно понял логику таблиц.

